# P3C3 Transformer les catégories textuelles

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OpenClassrooms-Student-Center/8063076-Initiez-vous-au-Machine-Learning/blob/master/notebooks/P3C3_transformer_categories.ipynb)

Dans ce notebook nous allons nous attaquer aux problèmes des catégories textuelles qu'il faut numériser pour qu'elles soient compatible avec les modèles.

Nous travaillons sur le dataset des arbres de Paris


Vous trouverez les versions suivantes du datasets

- sur le site [opendata de Paris](https://opendata.paris.fr/explore/dataset/les-arbres/information)
- version [brute](https://github.com/OpenClassrooms-Student-Center/8063076-Initiez-vous-au-Machine-Learning/blob/master/data/paris-arbres-2023-09-07.csv) extraite le 10-09-2023, le séparateur est ';'
- version [exploitable](https://github.com/OpenClassrooms-Student-Center/8063076-Initiez-vous-au-Machine-Learning/blob/master/data/paris-arbres-clean-2023-09-10.csv): les colonnes ont été renommées, le séparateur est ','


Ici nous travaillons sur cette version _exploitable_


In [38]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [39]:
dataset_url = "https://raw.githubusercontent.com/OpenClassrooms-Student-Center/8063076-Initiez-vous-au-Machine-Learning/master/data/paris-arbres-clean-2023-09-10.csv"
data = pd.read_csv(dataset_url)
df = data[data.libelle_francais == 'Platane'].copy()
df.shape

/var/folders/31/b9pwgxyd4rz86p81hzwb44bm0000gn/T/ipykernel_30819/3663920932.py:2: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dataset_url)


(42588, 16)

## Numérisation manuelle



In [40]:
categories = ['Adulte', 'Jeune (arbre)Adulte', 'Jeune (arbre)', 'Mature']

for n, categorie in zip(range(1,len(categories)+1), categories):
    print(categorie,"=>", n)
    df.loc[df.stade_de_developpement == categorie, 'stade_de_developpement'] = n

print()
print(df.stade_de_developpement.value_counts(dropna = False))

Adulte => 1
Jeune (arbre)Adulte => 2
Jeune (arbre) => 3
Mature => 4

stade_de_developpement
1      21620
2       8356
3       5916
NaN     3350
4       3346
Name: count, dtype: int64


## Avec OrdinalEncoder

de la librairie category_encoder

In [41]:
from category_encoders.ordinal import OrdinalEncoder

df = data[data.libelle_francais == 'Platane'].copy()


mapping =[ {'col': 'stade_de_developpement',
	'mapping': {
            	np.nan: 0,
            	'Jeune (arbre)': 1,
            	'Jeune (arbre)Adulte': 2,
            	'Adulte': 3,
            	'Mature': 4
            	}
        	} ]

encoder = OrdinalEncoder(mapping=mapping)
stade = encoder.fit_transform(df.stade_de_developpement)
stade.value_counts(dropna = False)

stade_de_developpement
3                         21620
2                          8356
1                          5916
0                          3350
4                          3346
Name: count, dtype: int64

## L'encodage one-hot

Encodons toutes les 9 valeurs de la variable domanialite

In [42]:
# enlevons les echantillons pour lesquels domanialite manque
df = df[~df.domanialite.isna()].copy()
df.reset_index(drop = True, inplace = True)

In [43]:
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder()
labels = enc.fit_transform(df.domanialite.values.reshape(-1, 1)).toarray()
labels.shape


(42588, 9)

In [44]:
labels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [45]:
new_columns = [ f"is_{col.lower()}" for col in df.domanialite.unique()   ]
new_columns


['is_alignement',
 'is_cimetiere',
 'is_jardin',
 'is_dasco',
 'is_peripherique',
 'is_djs',
 'is_dfpe',
 'is_dases',
 'is_dac']

In [46]:
df = pd.concat([df, 
        pd.DataFrame(columns = new_columns[:-1], data = labels[:, :-1])], 
        axis = 1)

In [53]:

for col in new_columns[:-1]:
    df[col] = df[col].astype(int)

In [54]:
df[['domanialite'] + new_columns[:-1]].sample(5, random_state = 88)

,domanialite,is_alignement,is_cimetiere,is_jardin,is_dasco,is_peripherique,is_djs,is_dfpe,is_dases
35830,Alignement,1,0,0,0,0,0,0,0
38285,Alignement,1,0,0,0,0,0,0,0
10698,Alignement,1,0,0,0,0,0,0,0
39917,CIMETIERE,0,1,0,0,0,0,0,0
12737,Alignement,1,0,0,0,0,0,0,0


### Omniprésence de zero
Regardons pour chaque nouvelle variable le taux de zero 



In [58]:
for col in new_columns[:-1]:
    print( f" {col}: \t {np.round(100.0*(1 - df[col].sum()/ df.shape[0]), 2)}% "  )

 is_alignement: 	 16.06% 
 is_cimetiere: 	 92.67% 
 is_jardin: 	 99.98% 
 is_dasco: 	 98.08% 
 is_peripherique: 	 99.99% 
 is_djs: 	 99.83% 
 is_dfpe: 	 99.57% 
 is_dases: 	 95.21% 


# Encodage Binaire

In [64]:
print(f"Il y a {len(data.espece.unique())} espèces d'arbres différentes")

Il y a 560 espèces d'arbres différentes


Le one hot encoding va créer 559 

In [59]:
from category_encoders.binary import BinaryEncoder
encoder = BinaryEncoder()
espece_bin = encoder.fit_transform(data.espece)

In [61]:
espece_bin.shape

(221201, 10)

In [62]:
espece_bin.head()

,espece_0,espece_1,espece_2,espece_3,espece_4,espece_5,espece_6,espece_7,espece_8,espece_9
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,1,0,1


Donc 10 nouvelles varibles sont nécessaires pour encoder 